# Lab Exercise 6: FSK-MSK

The purpose of this lab exercise is to simulate and analyze Frequency Shift Keying (FSK) and Minimum Shift Keying (MSK) systems. Students will compare coherent and non-coherent demodulation, calculate bit error rates, and examine the spectral properties of these modulation schemes. The exercise also includes an optional task to compare MSK and QPSK systems in terms of bit error rate and bandwidth.


## Setup


```{admonition} Live Code
Press the following button to make python code interactive. It will connect you to a kernel once it says "ready" (might take a bit, especially the first time it runs).
```

<div style="text-align: center;">
  <button title="Launch thebe" class="thebelab-button thebe-launch-button" onclick="initThebe()">Python Interactive Code</button>
</div>


#### Importing packages we will need later in Python


In [1]:
# Import necessary libraries
import numpy as np                  # For numerical computations
import matplotlib.pyplot as plt     # For plotting graphs
import random                       # For generating random numbers
import scipy.signal                 # For signal processing functions
from scipy.signal import welch, upfirdn, firwin, firwin2, lfilter  # Specific signal processing functions
import scipy.special                # For special mathematical functions
from scipy.special import erfc      # Complementary error function
import math                         # For mathematical operations
import scipy.signal as signal       # Alias for scipy.signal
from math import log, log2, sqrt    # Specific mathematical functions
import ipywidgets as widgets        # For creating interactive widgets in Jupyter notebooks
from ipywidgets import IntSlider, IntRangeSlider, FloatSlider, interactive, Layout, Dropdown, IntText, HBox, VBox, Output
from IPython.display import display, clear_output  # For displaying outputs in Jupyter notebooks
from commpy.channels import awgn    # For simulating Additive White Gaussian Noise channel
import time                         # For timing code execution
import warnings                     # For handling warnings

# Suppress warnings to keep the output clean
warnings.filterwarnings('ignore')

# Confirmation message
print("Libraries added successfully!")

Libraries added successfully!


## Part 1: Useful Code

`````{dropdown} Code 6.1
````{tab} Python
```python
def fsk_errors(bps, Nsymb, ns, EbNo):
    # Input parameters
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period

    # M frequencies in "coherent" distance (BR)
    f = fc + (BR/2) * (np.arange(1, M + 1) - (M + 1) / 2)

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((Nsymb, bps))

    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s = np.concatenate(s)

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        smi = []
        for fi in f:
            si = np.sin(2 * np.pi * fi * tk)
            smi.append(np.sum(sk * si))
        j = np.argmax(smi)
        xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr).reshape((Nsymb, bps))

    # count errors
    errors = np.sum(x != xr)
    return errors
```
````

````{tab} Matlab
```matlab
function errors=fsk_errors(bps,Nsymb,ns,EbNo)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Input parameters
% bps: bits per symbol, Nsymb: numb of simulated symbols
% ns: number of samples per symbol (oversampling)
% EbNo: normalized signal-to-noise ratio, in db
M=2^bps; % number of different symbols
BR=1; % Baud Rate
fc=2*M*BR; % RF frequency
%% Derived parameters
nb=bps*Nsymb; % number of simulated data bits
T=1/BR; % one symbol period
Ts=T/ns; % oversampling period
% M frequencies in "coherent" distance (BR)
f=fc+(BR/2)*((1:M)-(M+1)/2);
% Calculate the maximum frequency
fmax = max(f);
% Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax;
ns = ceil(Fs / BR) + 10;
% Recalculate the oversampling period
Ts = T / ns;
% awgn channel
SNR=EbNo+10*log10(bps)-10*log10(ns/2); % in db
% input data bits
y = randi([0 1], nb, 1); % Ensure integer dimensions
x = reshape(y, bps, [])';
t=[0:T:length(x(:,1))*T]'; % time vector on the T grid
tks=[0:Ts:T-Ts]';
%% FSK signal
s=[];
A=sqrt(2/T/ns);
for k=1:length(x(:,1))
    fk=f(bi2de(x(k,:))+1);
    tk=(k-1)*T+tks;
    s=[s; sin(2*pi*fk*tk)];
end
% add noise to the FSK (passband) signal
s=awgn(s,SNR, 'measured');
%% FSK receiver
% coherent demodulation
th=0;
xr=[];
for k=1:length(s)/ns
    tk=(k-1)*T+tks;
    sk=s((k-1)*ns+1:k*ns);
    smi=[];
    for i=1:M
        si=sin(2*pi*f(i)*tk);
        smi(i)=sum(sk.*si);
    end
    [m,j]=max(smi);
    xr=[xr;de2bi(j-1,bps)];
end
% count errors
err=not(x==xr);
errors=sum(sum(err));
end
```
````
`````


## Part 2: Code testing
Run code 6.1 of the notes (step-by-step, not by calling the function) for M=16 and
confirm the obtained number of errors for two different values of the normalized
signal-to-noise ratio


In [2]:
# Loading animation and completion checkmark HTML code for visual feedback
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border on top */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """

# Create HTML widgets to display the loading animation and elapsed time
loader_html1 = widgets.HTML(value=loading)
timer_html1 = widgets.HTML(value="Elapsed time: - seconds")

# Parameters for FSK modulation
bps = 4           # Bits per symbol (4 bits per symbol for 16-FSK)
Nsymb = 1000      # Number of symbols to simulate
ns = 80           # Number of samples per symbol (oversampling factor)

# Derived parameters
M = 2 ** bps      # Number of different symbols (M-ary FSK)
BR = 1            # Baud Rate (symbols per second)
fc = 2 * M * BR   # Carrier frequency for FSK modulation

# Further derived parameters
nb = bps * Nsymb  # Total number of bits to simulate
T = 1 / BR        # Symbol duration
Ts = T / ns       # Sampling period (time between samples)

# Generate M frequencies for coherent FSK modulation
f = fc + (BR / 2) * (np.arange(1, M + 1) - (M + 1) / 2)

# Calculate the maximum frequency used
fmax = np.max(f)

# Recalculate 'ns' to ensure the sampling frequency 'Fs' satisfies the Nyquist criterion
Fs = 2 * fmax                    # Sampling frequency must be greater than twice the max frequency
ns = int(np.ceil(Fs / BR)) + 10  # Adjust 'ns' accordingly and add some margin

# Recalculate the sampling period with the new 'ns'
Ts = T / ns

# Generate random input data bits
y = np.random.randint(0, 2, nb)   # Random bits (0 or 1)
x = y.reshape((Nsymb, bps))       # Reshape bits into symbols (each symbol has 'bps' bits)

# Time vectors for symbol intervals and oversampling
t = np.arange(0, len(x) * T, T)   # Time vector on the symbol grid
tks = np.arange(0, T, Ts)         # Time vector for oversampling within one symbol period

# Generate FSK signal
s = []                             # Initialize signal list
A = np.sqrt(2 / T / ns)            # Amplitude to normalize power
for k in range(len(x)):
    # Convert bits to symbol index
    symbol_bits = ''.join(map(str, x[k]))
    symbol_index = int(symbol_bits, 2)
    fk = f[symbol_index]           # Select frequency for the symbol
    tk = (k * T) + tks             # Time vector for the current symbol
    s.append(np.sin(2 * np.pi * fk * tk))  # Generate sine wave for the symbol
s = np.concatenate(s)              # Concatenate all symbol signals

def calculate_errors(EbNo_range):
    """
    Calculate the number of errors over a range of Eb/No values.

    Parameters:
    - EbNo_range: Tuple containing the start and end Eb/No values in dB
    """
    # Start timer and display loading animation
    loader_html1.value = loading
    start_time = time.time()

    clear_output(wait=True)  # Clear previous output
    EbNo_values = list(range(EbNo_range[0], EbNo_range[1] + 1))  # List of Eb/No values
    errors_list = []  # List to store the number of errors at each Eb/No

    for EbNo in EbNo_values:
        # Calculate Signal-to-Noise Ratio (SNR) in dB
        SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)

        # Add AWGN noise to the FSK signal
        noisy_signal = awgn(s, SNR)

        # FSK receiver implementation
        xr = []  # List to store received symbols
        for k in range(len(noisy_signal) // ns):
            tk = (k * T) + tks                      # Time vector for the current symbol
            sk = noisy_signal[k * ns:(k + 1) * ns]  # Extract the symbol from the noisy signal
            smi = []                                # List to store correlation values
            for fi in f:
                si = np.sin(2 * np.pi * fi * tk)    # Reference sine wave for frequency 'fi'
                smi.append(np.sum(sk * si))         # Correlate received signal with reference
            j = np.argmax(smi)                      # Find the index with the maximum correlation
            # Convert index to bits and append to received symbols
            xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
        xr = np.array(xr).reshape((Nsymb, bps))     # Reshape received symbols

        # Count the number of bit errors
        errors = np.sum(x != xr)
        errors_list.append(errors)

    # Plot the number of errors versus Eb/No
    plt.figure(figsize=(10, 6))
    plt.plot(EbNo_values, errors_list, marker='o', linestyle='-', markersize=8)
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Number of Errors')
    plt.title('Number of Errors vs. Eb/No')
    plt.grid(True)
    plt.show()
    
    # Display elapsed time and completion checkmark
    elapsed_time = time.time() - start_time
    timer_html1.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html1.value = done

# Create a range slider widget for Eb/No values
EbNo_slider = IntRangeSlider(
    value=[0, 20],          # Default range from 0 to 20 dB
    min=0,                  # Minimum value
    max=20,                 # Maximum value
    step=1,                 # Step size
    description='EbNo (dB):',
    continuous_update=False,
    layout=Layout(width='99%')
)

# Group the loader and timer widgets
loader_timer_box = widgets.VBox(
    [loader_html1, timer_html1],
    layout=widgets.Layout(margin='0 0 0 20px', width='auto')
)

# Create a VBox for input widgets
input_widgets = widgets.VBox(
    [EbNo_slider],
    layout=widgets.Layout(flex='1 1 auto', width='auto')
)

# Combine input widgets and loader/timer into an HBox
ui = widgets.HBox(
    [input_widgets, loader_timer_box],
    layout=widgets.Layout(align_items='center')
)

# Create the interactive output
out = widgets.interactive_output(calculate_errors, {'EbNo_range': EbNo_slider})

# Display the UI and output
clear_output(wait=True)  # Clear previous output
display(ui, out)

Output()

## Part 3: Non-coherent demodulation

Complete the above code (code 6.1) in order to also simulate the non-coherent FSK

```{tip}
Add random phase to the received signal, before demodulation.
```


`````{dropdown} Code 6.2
````{tab} Python
```python
def fsk_errors_non_coh(bps, Nsymb, ns, EbNo):
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)  # M frequencies

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # AWGN channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)  # Ensure integer dimensions
    x = y.reshape((-1, bps))
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)

    # FSK signal
    s = np.array([])
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int("".join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s = np.concatenate((s, np.sin(2 * np.pi * fk * tk)))

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    # Non-coherent demodulation
    xr = np.array([])
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        sm = []
        for i in range(M):
            si = np.sin(2 * np.pi * (f[i] * tk))
            sq = np.cos(2 * np.pi * (f[i] * tk))
            smi = np.sum(sk * si)
            smq = np.sum(sk * sq)
            sm.append(np.sqrt(smi ** 2 + smq ** 2))
        j = np.argmax(sm)
        xr = np.concatenate((xr, np.array(list(np.binary_repr(j, width=bps)), dtype=int)))

    # count errors
    x_reshaped = x.reshape(-1)
    xr_reshaped = xr.reshape(-1)
    err = np.not_equal(x_reshaped, xr_reshaped)
    errors = np.sum(err)
    return errors
```
````

````{tab} Matlab
```matlab
function errors = fsk_errors_non_coh(bps, Nsymb, ns, EbNo)
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %% Input parameters
    % bps: bits per symbol, Nsymb: number of simulated symbols
    % ns: number of samples per symbol (oversampling)
    % EbNo: normalized signal-to-noise ratio, in dB
    M = 2^bps; % number of different symbols
    BR = 1; % Baud Rate
    fc = 2 * M * BR; % RF frequency
    
    %% Derived parameters
    nb = bps * Nsymb; % number of simulated data bits
    T = 1 / BR; % one symbol period
    Ts = T / ns; % oversampling period
    % M frequencies in "non-coherent" distance (BR)
    f = fc + BR * ((1:M) - (M + 1) / 2);
    % Calculate the maximum frequency
    fmax = max(f);

    % Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax;
    ns = ceil(Fs / BR) + 10;

    % Recalculate the oversampling period
    Ts = T / ns;
    % AWGN channel
    SNR = EbNo + 10 * log10(bps) - 10 * log10(ns / 2); % in dB
    % input data bits
    y = randi([0 1], nb, 1); % Ensure integer dimensions
    x = reshape(y, bps, [])';
    t = [0:T:length(x(:,1)) * T]'; % time vector on the T grid
    tks = [0:Ts:T-Ts]';
    
    %% FSK signal
    s = [];
    A = sqrt(2 / T / ns);
    for k = 1:length(x(:,1))
        fk = f(bi2de(x(k,:)) + 1);
        tk = (k - 1) * T + tks;
        s = [s; sin(2 * pi * fk * tk)];
    end
    
    % add noise to the FSK (passband) signal
    s = awgn(s, SNR, 'measured');
    
    %% FSK receiver
    % Non-coherent demodulation
    th = 0;
    xr = [];
    for k = 1:length(s) / ns
        tk = (k - 1) * T + tks;
        sk = s((k - 1) * ns + 1:k * ns);
        sm = [];
        for i = 1:M
            si = sin(2 * pi * (f(i) * tk + th));
            sq = cos(2 * pi * (f(i) * tk + th));
            smi = sum(sk .* si);
            smq = sum(sk .* sq);
            sm(i) = sqrt(smi^2 + smq^2);
        end
        [m, j] = max(sm);
        xr = [xr; de2bi(j - 1, bps)];
    end
    
    % count errors
    err = not(x == xr);
    errors = sum(sum(err));
end
```
````
`````


## Part 3:

Use your new function to simulate a 16-FSK system and plot the Pb vs Eb/No curves
for coherent and non-coherent demodulation (theoretical and from simulation).


In [3]:
# Loading animation and completion checkmark HTML code
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border on top */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """
loader_html2 = widgets.HTML(value=loading)
timer_html2 = widgets.HTML(value="Elapsed time: - seconds")

def fsk_errors(bps, Nsymb, ns, EbNo):
    """
    Simulate coherent FSK and calculate the number of bit errors.

    Parameters:
    - bps: Bits per symbol
    - Nsymb: Number of symbols
    - ns: Samples per symbol
    - EbNo: Energy per bit to noise power spectral density ratio in dB

    Returns:
    - errors: Number of bit errors
    """
    # Input parameters
    M = 2 ** bps              # Modulation order
    BR = 1                    # Baud Rate
    fc = 2 * M * BR           # Carrier frequency

    # Derived parameters
    nb = bps * Nsymb          # Total number of bits
    T = 1 / BR                # Symbol duration
    Ts = T / ns               # Sampling period

    # Generate frequencies for coherent FSK
    f = fc + (BR / 2) * (np.arange(1, M + 1) - (M + 1) / 2)

    # Calculate the maximum frequency and adjust 'ns' accordingly
    fmax = np.max(f)
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10
    Ts = T / ns

    # Calculate SNR in dB
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)

    # Generate random input bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((Nsymb, bps))

    # Time vectors
    t = np.arange(0, len(x) * T, T)
    tks = np.arange(0, T, Ts)

    # Generate FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        symbol_bits = ''.join(map(str, x[k]))
        symbol_index = int(symbol_bits, 2)
        fk = f[symbol_index]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s = np.concatenate(s)

    # Add AWGN noise to the signal
    s = awgn(s, SNR)

    # FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        smi = []
        for fi in f:
            si = np.sin(2 * np.pi * fi * tk)
            smi.append(np.sum(sk * si))
        j = np.argmax(smi)
        xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr).reshape((Nsymb, bps))

    # Count errors
    errors = np.sum(x != xr)
    return errors

def fsk_errors_non_coh(bps, Nsymb, ns, EbNo):
    """
    Simulate non-coherent FSK and calculate the number of bit errors.

    Parameters:
    - bps: Bits per symbol
    - Nsymb: Number of symbols
    - ns: Samples per symbol
    - EbNo: Energy per bit to noise power spectral density ratio in dB

    Returns:
    - errors: Number of bit errors
    """
    M = 2 ** bps            # Modulation order
    BR = 1                  # Baud Rate
    fc = 2 * M * BR         # Carrier frequency

    # Derived parameters
    nb = bps * Nsymb
    T = 1 / BR
    Ts = T / ns

    # Generate frequencies for non-coherent FSK
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)

    # Adjust 'ns' based on maximum frequency
    fmax = np.max(f)
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10
    Ts = T / ns

    # Calculate SNR in dB
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)

    # Generate random input bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((-1, bps))
    t = np.arange(0, len(x) * T, T)
    tks = np.arange(0, T, Ts)

    # Generate FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        symbol_bits = ''.join(map(str, x[k]))
        symbol_index = int(symbol_bits, 2)
        fk = f[symbol_index]
        tk = (k * T) + tks
        s = np.concatenate((s, np.sin(2 * np.pi * fk * tk)))

    # Add AWGN noise
    s = awgn(s, SNR)

    # Non-coherent FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        sm = []
        for i in range(M):
            si = np.sin(2 * np.pi * f[i] * tk)
            sq = np.cos(2 * np.pi * f[i] * tk)
            smi = np.sum(sk * si)
            smq = np.sum(sk * sq)
            sm.append(np.sqrt(smi ** 2 + smq ** 2))
        j = np.argmax(sm)
        xr.extend([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr)

    # Reshape and count errors
    x_reshaped = x.reshape(-1)
    xr_reshaped = xr[:len(x_reshaped)]
    errors = np.sum(x_reshaped != xr_reshaped)
    return errors

def simulate_ber(bps, Nsymb, ns, EbNo_values, coherent=True):
    """
    Simulate Bit Error Rate (BER) over a range of Eb/No values.

    Parameters:
    - bps: Bits per symbol
    - Nsymb: Number of symbols
    - ns: Samples per symbol
    - EbNo_values: List of Eb/No values in dB
    - coherent: Boolean indicating coherent or non-coherent detection

    Returns:
    - ber: List of BER values corresponding to Eb/No values
    """
    ber = []
    for EbNo in EbNo_values:
        if coherent:
            errors = fsk_errors(bps, Nsymb, ns, EbNo)
        else:
            errors = fsk_errors_non_coh(bps, Nsymb, ns, EbNo)
        ber.append(errors / (Nsymb * bps))
    return ber


# Define the BER values directly inside the functions
def theoretical_ber_coh(EbNo, M):
    """
    Return theoretical BER values for coherent FSK.

    Parameters:
    - EbNo: Array of Eb/No values in linear scale
    - M: Modulation order

    Returns:
    - BER: Array of BER values
    """
    if M == 2:
        return np.array([0.15866, 0.13093, 0.10403, 0.078896, 0.056495,
    0.037679, 0.023007, 0.012587, 0.0060044, 0.0024133,
    0.0007827, 0.00019399, 3.4303e-05, 3.9692e-06, 2.6951e-07])
    elif M == 4:
        return np.array([0.11814, 0.087789, 0.060786, 0.038512, 0.021824, 
    0.010751, 0.0044428, 0.0014733, 0.00037102, 6.6229e-05, 
    7.6892e-06, 5.2118e-07, 1.7997e-08, 2.6653e-10, 1.362e-12])
    elif M == 8:
        return np.array([0.10227, 0.067834, 0.041318, 0.022024, 0.0099156, 
    0.0036087, 0.0010058, 0.00020086, 2.6486e-05, 2.0836e-06, 
    8.6119e-08, 1.5924e-09, 1.074e-11, 2.0391e-14, 7.8524e-18])
    elif M == 16:
        return np.array([0.089859, 0.055663, 0.029957, 0.013469, 0.004819,
    0.001293, 0.00024205, 2.897e-05, 1.9921e-06, 6.8928e-08,
    1.0145e-09, 5.1257e-12, 6.7629e-15, 1.6453e-18, 4.6157e-23])
    elif M == 32:
        return np.array([0.082719, 0.047105, 0.022469, 0.0085348, 0.0024266,
    0.00047917, 6.0083e-05, 4.2975e-06, 1.5388e-07, 2.3425e-09,
    1.2294e-11, 1.6992e-14, 4.3826e-18, 1.3266e-22, 2.1688e-28])
    else:
        raise ValueError("Unsupported value of M for coherent case")


def theoretical_ber_non_coh(EbNo, M):
    """
    Return theoretical BER values for non-coherent FSK.

    Parameters:
    - EbNo: Array of Eb/No values in linear scale
    - M: Modulation order

    Returns:
    - BER: Array of BER values
    """
    if M == 2:
        return np.array([0.30327, 0.26644, 0.22637, 0.18438, 0.1424, 0.10287, 
    0.068311, 0.0408, 0.021324, 0.0094212, 0.003369, 
    0.0009231, 0.00018089, 2.3244e-05, 1.7558e-06])
    elif M == 4:
        return np.array([0.22934, 0.18475, 0.13987, 0.097719, 0.061557, 
    0.033946, 0.01579, 0.0059139, 0.0016837, 0.00033939, 
    4.4371e-05, 3.3753e-06, 1.3045e-07, 2.1593e-09, 1.233e-11])
    elif M == 8:
        return np.array([0.19472, 0.14559, 0.099187, 0.059806, 0.030757, 
    0.012878, 0.0041438, 0.00095467, 0.00014449, 1.2945e-05, 
    6.0428e-07, 1.2541e-08, 9.4638e-11, 2.0092e-13, 8.6607e-17])
    elif M == 16:
        return np.array([0.17469, 0.12169, 0.074737, 0.038861, 0.01625, 
    0.005127, 0.0011288, 0.00015786, 1.2538e-05, 4.943e-07, 
    8.2001e-09, 4.6432e-11, 6.8517e-14, 1.8682e-17, 6.0826e-22])
    elif M == 32:
        return np.array([0.16103, 0.10471, 0.058014, 0.025984, 0.0088058, 
    0.0020817, 0.00031127, 2.6219e-05, 1.0859e-06, 1.8789e-08, 
    1.1086e-10, 1.7154e-13, 4.9582e-17, 1.737e-21, 4.2722e-27])
    else:
        raise ValueError("Unsupported value of M for non-coherent case")


def update_plot(bps):
    """
    Update the BER plot based on the selected bits per symbol.

    Parameters:
    - bps: Bits per symbol
    """
    # Start timer and display loading animation
    loader_html2.value = loading
    start_time = time.time()

    Nsymb = 2000             # Number of symbols
    ns = 100                 # Samples per symbol
    EbNo_dB_sim = np.arange(0, 10, 2)    # Eb/No values for simulation
    EbNo_dB_theory = np.arange(0, 15, 1) # Eb/No values for theoretical curves

    EbNo_sim = 10 ** (EbNo_dB_sim / 10)        # Convert dB to linear scale
    EbNo_theory = 10 ** (EbNo_dB_theory / 10)
    M = 2 ** bps                                # Modulation order

    # Simulate BER for coherent and non-coherent detection
    ber_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=True)
    ber_non_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=False)

    # Get theoretical BER values
    ber_coh_theory = theoretical_ber_coh(EbNo_theory, M)
    ber_non_coh_theory = theoretical_ber_non_coh(EbNo_theory, M)

    # Plot BER curves
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbNo_dB_sim, ber_coh_sim, 'o', label=f'Simulated Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_sim, ber_non_coh_sim, 's', label=f'Simulated Non-Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_coh_theory, label=f'Theoretical Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_non_coh_theory, label=f'Theoretical Non-Coherent {M}-FSK')

    plt.title(f'{M}-FSK System')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.grid(True, which='both')
    plt.legend()
    plt.show()

    # Display elapsed time and completion checkmark
    elapsed_time = time.time() - start_time
    timer_html2.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html2.value = done

# Dropdown widget for selecting bits per symbol
bps_dropdown = Dropdown(
    options=[('2-FSK', 1), ('4-FSK', 2), ('8-FSK', 3), ('16-FSK', 4), ('32-FSK', 5)],
    value=2,
    description='M-FSK:',
    style={'description_width': 'initial'}, 
    continuous_update=False
)

# Group the loader and timer widgets
loader_timer_box = widgets.VBox(
    [loader_html2, timer_html2],
    layout=widgets.Layout(margin='0 0 0 20px', width='auto')
)

# Input widgets container
input_widgets = widgets.VBox([bps_dropdown])

# Combine input widgets and loader/timer into an HBox
ui = widgets.HBox(
    [input_widgets, loader_timer_box],
    layout=widgets.Layout(align_items='center')
)

# Create interactive output
out = widgets.interactive_output(update_plot, {'bps': bps_dropdown})

# Display UI and output
clear_output(wait=True)
display(ui, out)

Output()

## Part 4:

Plot the spectrum of the bandpass signal of question 3.


`````{dropdown} Solution
````{tab} Python - Coherent
```python
# Input parameters
bps = 4
Nsymb = 10000
EbNo = 8

# Derived parameters
M = 2 ** bps  # number of different symbols
BR = 1  # Baud Rate
fc = 2 * M * BR  # RF frequency
ns = 80  # samples per symbol
nb = bps * Nsymb  # number of simulated data bits
f = fc + (BR / 2) * ((np.arange(1, M + 1)) - (M + 1) / 2)

# Calculate the maximum frequency
fmax = np.max(f)

# Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax
ns = int(np.ceil(Fs / BR)) + 10

# Recalculate the oversampling period
Ts = T / ns

# awgn channel
SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

# input data bits
y = np.random.randint(0, 2, nb)
x = np.reshape(y, (len(y) // bps, bps))

# time vectors
t = np.arange(0, len(x) * T, T)  # time vector on the T grid
tks = np.arange(0, T, Ts)  # oversampling time vector

# FSK signal generation
s = []
A = np.sqrt(2 / T / ns)

for k in range(len(x)):
    fk = f[int(''.join(map(str, x[k])), 2)]
    tk = (k * T) + tks
    s.append(np.sin(2 * np.pi * fk * tk))

s = np.concatenate(s)

# Plotting power spectral density with higher resolution
frequencies, Pxx = welch(s, fs=ns*BR, nperseg=50000)
plt.figure()
plt.semilogy(frequencies, Pxx, linewidth=0.5)
plt.title("FSK Coherent")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density (dB/Hz)")
plt.grid(True)
plt.show()
```
````

````{tab} Matlab - Coherent
```matlab
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Input parameters
clear all;
bps=4;Nsymb=10000;EbNo=8;
% bps: bits per symbol, Nsymb: numb of simulated symbols
% ns: number of samples per symbol (oversampling)
% EbNo: normalized signal-to-noise ratio, in db
M=2^bps; % number of different symbols
BR=1; % Baud Rate
fc=2*M*BR; % RF frequency
ns=80; % ns samples per symbol ώστε Fs=ns*Fd >=2*fmax
%% Derived parameters
nb=bps*Nsymb; % number of simulated data bits
% M frequencies in coherent distance (BR/2)
f=fc+(BR/2)*((1:M)-(M+1)/2);
fmax = max(f); % Calculate the maximum frequency
% Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax;
ns = ceil(Fs / BR) + 10;
% Recalculate the oversampling period
Ts = T / ns;
% awgn channel
SNR=EbNo+10*log10(bps)-10*log10(ns/2); % in db
% input data bits
y=randi([0 1],1,nb);%
x=reshape(y,bps,length(y)/bps)';
t=[0:T:length(x(:,1))*T]'; % time vector on the T grid
tks=[0:Ts:T-Ts]';
%% FSK signal
s=[];
A=sqrt(2/T/ns);
%th=pi*rand(M,1);
for k=1:length(x(:,1))
fk=f(bi2de(x(k,:))+1);
tk=(k-1)*T+tks;
s=[s; sin(2*pi*fk*tk)]; % th(bi2de(x(k,:))+1))];
end

figure;pwelch(s,[],[],[],ns*BR);title("fsk coherent");
```
````

````{tab} Python - Non-coherent
```python
# Input parameters
bps = 4
Nsymb = 10000
EbNo = 8

# Derived parameters
M = 2 ** bps  # number of different symbols
BR = 1  # Baud Rate
fc = 2 * M * BR  # RF frequency
ns = 90  # samples per symbol
nb = bps * Nsymb  # number of simulated data bits
T = 1 / BR  # one symbol period
Ts = T / ns  # oversampling period

# Frequencies in "non-coherent" distance
f = fc + BR * ((np.arange(1, M + 1)) - (M + 1) / 2)

# Calculate the maximum frequency
fmax = np.max(f)

# Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax
ns = int(np.ceil(Fs / BR)) + 10

# Recalculate the oversampling period
Ts = T / ns

# awgn channel
SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

# input data bits
y = np.random.randint(0, 2, nb)
x = np.reshape(y, (len(y) // bps, bps))

# time vectors
t = np.arange(0, len(x) * T, T)  # time vector on the T grid
tks = np.arange(0, T, Ts)  # oversampling time vector

# FSK signal generation
s = []
A = np.sqrt(2 / T / ns)

for k in range(len(x)):
    fk = f[int(''.join(map(str, x[k])), 2)]
    tk = (k * T) + tks
    s.append(np.sin(2 * np.pi * fk * tk))

s = np.concatenate(s)

# Plotting power spectral density with high resolution and thinner line
frequencies, Pxx = welch(s, fs=ns*BR, nperseg=50000)
plt.figure()
plt.semilogy(frequencies, Pxx, linewidth=0.5)
plt.title("Non Coherent FSK")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density (dB/Hz)")
plt.grid(True)
plt.show()
```
````

````{tab} Matlab - Non-coherent
```matlab
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%% Input parameters
% bps: bits per symbol, Nsymb: numb of simulated symbols
% ns: number of samples per symbol (oversampling)
% EbNo: normalized signal-to-noise ratio, in db
clear all;
bps=4;Nsymb=10000;EbNo=8;
M=2^bps; % number of different symbols
BR=1; % Baud Rate
fc=2*M*BR; % RF frequency
ns=90; % ns samples per symbol ώστε Fs=ns*Fd >=2*fmax, 
%% Derived parameters
nb=bps*Nsymb; % number of simulated data bits
T=1/BR; % one symbol period-duration
f=fc+BR*((1:M)-(M+1)/2);% M frequencies in "non-coherent" distance (BR)
fmax = max(f); % Calculate the maximum frequency
% Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax;
ns = ceil(Fs / BR) + 10;
% Recalculate the oversampling period
Ts = T / ns;
% awgn channel
SNR=EbNo+10*log10(bps)-10*log10(ns/2); % in db
% input data bits
y=randi([0 1],1,nb);%
x=reshape(y,bps,length(y)/bps)';
t=[0:T:length(x(:,1))*T]'; % time vector on the T grid
tks=[0:Ts:T-Ts]';
%% FSK signal
s=[];
A=sqrt(2/T/ns);
for k=1:length(x(:,1))
fk=f(bi2de(x(k,:))+1);
tk=(k-1)*T+tks;
s=[s; sin(2*pi*fk*tk)]; % th(bi2de(x(k,:))+1))];
end
figure;pwelch(s,[],[],[],ns*BR);title("Non Coherent fsk");
```
````
`````

In [4]:
# Loading animation and completion checkmark HTML code
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey border */
                                     border-top: 12px solid #01cc97; /* Blue border on top */
                                     border-radius: 50%;             /* Circular shape */
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite; /* Spinning animation */'>
        </div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }     /* Start rotation */
        100% { transform: rotate(360deg); } /* End rotation */
    }
    </style>
    """
done = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>  <!-- Checkmark symbol -->
    </div>
    """
loader_html3 = widgets.HTML(value=loading)
timer_html3 = widgets.HTML(value="Elapsed time: - seconds")

def update_plot(fsklvl, Nsymb, EbNo):
    """
    Plot the power spectral density for coherent and non-coherent FSK.

    Parameters:
    - fsklvl: FSK levels (e.g., 2, 4, 8, etc.)
    - Nsymb: Number of symbols
    - EbNo: Energy per bit to noise power spectral density ratio in dB
    """
    # Start timer and display loading animation
    loader_html3.value = loading
    start_time = time.time()

    clear_output(wait=True)
    
    bps = int(math.log2(fsklvl))  # Bits per symbol
    M = 2 ** bps                  # Modulation order
    BR = 1                        # Baud Rate
    T = 1 / BR                    # Symbol duration
    fc = 2 * M * BR               # Carrier frequency
    ns = 80                       # Samples per symbol
    nb = bps * Nsymb              # Total number of bits
    f = fc + (BR / 2) * (np.arange(1, M + 1) - (M + 1) / 2)  # Frequencies for coherent FSK

    # Adjust 'ns' based on maximum frequency
    fmax = np.max(f)
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10
    Ts = T / ns

    # Calculate SNR in dB
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)

    # Generate random input bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((len(y) // bps, bps))

    # Time vectors
    t = np.arange(0, len(x) * T, T)
    tks = np.arange(0, T, Ts)

    # Generate coherent FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        symbol_bits = ''.join(map(str, x[k]))
        symbol_index = int(symbol_bits, 2)
        fk = f[symbol_index]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s_coherent = np.concatenate(s)

    # Compute power spectral density
    frequencies_coherent, Pxx_coherent = welch(s_coherent, fs=ns * BR, nperseg=50000, noverlap=25000)

    # Parameters for non-coherent FSK
    ns = 90
    Ts = T / ns
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)  # Frequencies for non-coherent FSK
    fmax = np.max(f)
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10
    Ts = T / ns

    # Generate non-coherent FSK signal
    s = []
    for k in range(len(x)):
        symbol_bits = ''.join(map(str, x[k]))
        symbol_index = int(symbol_bits, 2)
        fk = f[symbol_index]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s_non_coherent = np.concatenate(s)

    # Compute power spectral density
    frequencies_non_coherent, Pxx_non_coherent = welch(s_non_coherent, fs=ns * BR, nperseg=50000)

    # Plot PSDs
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))

    axs[0].semilogy(frequencies_coherent, Pxx_coherent, linewidth=0.5)
    axs[0].set_title("Coherent FSK")
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_ylabel("Power Spectral Density (dB/Hz)")
    axs[0].grid(True)

    axs[1].semilogy(frequencies_non_coherent, Pxx_non_coherent, linewidth=0.5)
    axs[1].set_title("Non-Coherent FSK")
    axs[1].set_xlabel("Frequency (Hz)")
    axs[1].set_ylabel("Power Spectral Density (dB/Hz)")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()
    
    # Display elapsed time and completion checkmark
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html3.value = done

# Widgets for user input
fsklvl_dropdown = Dropdown(
    options=[2, 4, 8, 16, 32],
    value=4,
    description='FSK Levels:',
    style={'description_width': 'initial'},
    layout=Layout(flex='1 1 auto', width='auto'),
    continuous_update=False
)
Nsymb_slider = IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step=1000,
    description='Number of Symbols:',
    style={'description_width': 'initial'},
    layout=Layout(flex='1 1 auto', width='auto'),
    continuous_update=False
)
EbNo_slider = IntSlider(
    value=8,
    min=0,
    max=20,
    step=1,
    description='Eb/No (dB):',
    style={'description_width': 'initial'},
    layout=Layout(flex='1 1 auto', width='auto'),
    continuous_update=False
)

# Group the loader and timer widgets
loader_timer_box = widgets.VBox(
    [loader_html3, timer_html3],
    layout=widgets.Layout(margin='0 0 0 20px', width='auto')
)

# Input widgets container
input_widgets = widgets.VBox(
    [fsklvl_dropdown, Nsymb_slider, EbNo_slider],
    layout=widgets.Layout(flex='1 1 auto', width='auto')
)

# Combine input widgets and loader/timer into an HBox
ui = widgets.HBox(
    [input_widgets, loader_timer_box],
    layout=widgets.Layout(align_items='center')
)

# Create interactive output
out = widgets.interactive_output(update_plot, {'fsklvl': fsklvl_dropdown, 'Nsymb': Nsymb_slider, 'EbNo': EbNo_slider})

# Display UI and output
clear_output(wait=True)
display(ui, out)

Output()

## Part 5

```{note}
Optional. Bonus 10% to Lab grade
```


### Step 1:

Based on Code 6.2 of the notes, simulate an MSK transmission system on a
bandpass channel with a center frequency of 8 MHz and a transmission rate
of 2 Mbps. Plot the spectrum of the bandpass signal and calculate
(theoretically and by simulation) the BER, when Eb/No=10db.


`````{dropdown} Code 6.3
````{tab} Python
```python
# MSK error function
def msk_errors(Nbits, nsamp, EbNo):
    n = Nbits  # number of data bits
    R = 2000000  # bit rate
    fc = 8000000  # carrier frequency
    ns = nsamp  # oversampling factor

    # AWGN channel
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB
    T = 1 / R  # 1-bit period (= basic period)
    Ts = T / ns  # sampling frequency
    fss = 1/Ts

    # Input sequence
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y

    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time grid
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    ## MSK TRANSMITTER
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)  # in-phase component
    xs_q = np.sin(theta)  # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid `ts`
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    bi = 1
    xr_1 = 1
    xr = np.zeros(n)
    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # matched-filter pulse
        gmq = -gmi  # =sin(pi/2/T*Ts*lq);
        bi_1 = bi
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        xr[k] = bi_1 * bq
        xr[k+1] = bi * bq
        xr_1 = xr[k + 1]

    xr = xr.reshape(-1)
    err = np.not_equal(x, xr)
    errors = np.sum(err)
    return errors / Nbits

# MSK error function with precoding
def msk_errors_precoding(Nbits, nsamp, EbNo):
    n = Nbits  # number of data bits
    R = 2000000  # bit rate
    fc = 8000000  # carrier frequency
    ns = nsamp  # oversampling factor

    # AWGN channel
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB
    T = 1 / R  # 1-bit period (= basic period)
    Ts = T / ns  # sampling frequency
    fss = 1/Ts

    # Input sequence
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y
    x[0] = 1
    for i in range(1, len(y)):
        x[i] = y[i] * x[i-1]  # Apply precoding rule

        

    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time grid
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    ## MSK TRANSMITTER
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)  # in-phase component
    xs_q = np.sin(theta)  # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid ts
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    # Updated MSK decoding for precoded bits with recursive logic
    bi = 1
    xr_1 = 1  # Initialize previous decoded bit (xr_1)
    xr = np.zeros(n)  # Array to store decoded bits

    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        
        # Matched filter output (to match MSK modulation characteristics)
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # In-phase matched filter pulse
        gmq = -gmi  # Quadrature matched-filter pulse (sin is negative of cosine)
        
        # Save previous in-phase matched filter output
        bi_1 = bi
        
        # Decode in-phase (I) and quadrature (Q) components
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        
        # Apply recursive decoding rule for precoded MSK
        xr[k] = bi_1 * bq  # Decode the k-th bit
        xr[k+1] = bi * bq  # Decode the (k+1)-th bit
        
        # Update the previously decoded bit (xr_1)
        xr_1 = xr[k+1]

    xr = xr.reshape(-1)
    err = np.not_equal(y, xr)
    errors = np.sum(err)
    return 0.5*errors / Nbits
```
````
````{tab} Matlab
```matlab
function errors=msk_errors(Nbits,nsamp,EbNo)
% Defined parameters
n=Nbits; % number of data bits
R=270833; % bit rate
fc=3*R; % carrier frequency
ns=nsamp; % oversampling factor
%
% δίαυλος awgn
SNR=EbNo-10*log10(ns/2); % in db
% Derived parameters
T=1/R; % 1-bit period (= basic period)
Ts=T/ns; % sampling frequency
% input sequence
y=[1;sign(rand(n-1,1)-0.5)]; % random numbers, -1 or 1
%
% precoding
x(1)=1;
for i=2:length(y)
    x(i)=y(i)*y(i-1);
end
x=x';
g=ones(ns,1);
xx=conv(upsample(x,ns),g); % NRZ polar pulse train samples
% time grid
ts=[0:Ts:length(xx)*Ts]'; % of length ns*(n+1)
%
%% MSK TRANSMITTER
xs=xx;
theta=cumsum(xs)*pi/2/ns;
xs_i=cos(theta); % in-phase component
xs_i=[xs_i; xs_i(length(xs_i))]; % one-sample extension
xs_q=sin(theta); % quadrature component
xs_q=[xs_q; xs_q(length(xs_q))]; % one-sample extension
% Modulation
s=xs_i.*cos(2*pi*fc*ts)-xs_q.*sin(2*pi*fc*ts);
% addition of noise
s=awgn(s,SNR, 'measured');
%% MSK RECEIVER
xs_i=s.*cos(2*pi*fc*ts);
xs_q=-s.*sin(2*pi*fc*ts);
% LP (Parks-McClellan)filter
f1=0.75/ns; f2=4*f1;
order=4*ns;
fpts=[0 f1 f2 1];
mag=[1 1 0 0];
wt=[1 1];
b = firpm(order,fpts,mag,wt);
a=1;
len=length(xs_i);
dummy=[xs_i;zeros(order,1)];
dummy1=filter(b,a,dummy);
delay=order/2; % Try with delay=0!
xs_i=dummy1(delay+(1:len));
dummy=[xs_q;zeros(order,1)];
dummy1=filter(b,a,dummy);
delay=order/2;
xs_q=dummy1(delay+(1:len));
bi=1; xr_1=1;
for k=1:2:n-1
    li=(k*ns+1:(k+2)*ns)';
    lq=((k-1)*ns+1:(k+1)*ns)';
    xi=xs_i(li);
    xq=xs_q(lq);
    gmi=cos(pi/2/T*Ts*li); % matched-filter pulse
    gmq=-gmi; % =sin(pi/2/T*Ts*lq);
    bi_1=bi;
    bi=sign(sum(xi.*gmi));
    bq=sign(sum(xq.*gmq));
    % without precoding: de-comment next 2 lines
    % xr(k)=bi_1*bq;
    % xr(k+1)=bi*bq;
    % with precoding: de-comment next 2 lines
    xr(k)=xr_1*bi_1*bq;
    xr(k+1)=xr(k)*bi*bq;
    xr_1=xr(k+1);
end
xr=xr';
% err=not(x==xr);
err=not(y==xr); % de-comment, with pre-coding
errors=sum(err);
end
```
````
`````

In [6]:
# Loading animation and completion checkmark HTML code
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html4 = widgets.HTML(
  value=loading
)
timer_html4 = widgets.HTML(
    value="Elapsed time: - seconds"
)


# Function that simulates Additive White Gaussian Noise (AWGN)
def awgn(signal, SNR):
    """
    Additive White Gaussian Noise (AWGN) channel simulation.

    Parameters:
    - signal: Input signal
    - SNR: Signal-to-Noise Ratio in dB

    Returns:
    - noisy_signal: Signal with added AWGN noise
    """
    snr_linear = 10 ** (SNR / 10)
    signal_power = np.mean(signal ** 2)
    noise_power = signal_power / snr_linear
    noise = np.sqrt(noise_power) * np.random.normal(size=signal.shape)
    return signal + noise


# MSK error function
def msk_errors(Nbits, nsamp, EbNo):
    """
    Simulate MSK modulation and calculate BER without precoding.

    Parameters:
    - Nbits: Number of bits
    - nsamp: Oversampling factor
    - EbNo: Energy per bit to noise power spectral density ratio in dB

    Returns:
    - BER: Bit Error Rate
    """
    n = Nbits             # Number of data bits
    R = 2000000           # Bit rate (2 Mbps)
    fc = 8000000          # Carrier frequency (8 MHz)
    ns = nsamp            # Oversampling factor
    T = 1 / R             # Bit duration
    Ts = T / ns           # Sampling interval
    fss = 1 / Ts          # Sampling frequency

    # Calculate SNR in dB
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB

    # Generate random input bits (-1 or 1)
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y

    # Generate NRZ polar pulse train samples
    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time vector
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    # MSK Transmitter
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)    # in-phase component
    xs_q = np.sin(theta)    # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid `ts`
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    bi = 1
    xr_1 = 1
    xr = np.zeros(n)
    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # matched-filter pulse
        gmq = -gmi  # =sin(pi/2/T*Ts*lq);
        bi_1 = bi
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        xr[k] = bi_1 * bq
        xr[k+1] = bi * bq
        xr_1 = xr[k + 1]

    xr = xr.reshape(-1)
    err = np.not_equal(x, xr)
    errors = np.sum(err)
    return errors / Nbits


# MSK error function with precoding
def msk_errors_precoding(Nbits, nsamp, EbNo):
    """
    Simulate MSK modulation and calculate BER with precoding.

    Parameters:
    - Nbits: Number of bits
    - nsamp: Oversampling factor
    - EbNo: Energy per bit to noise power spectral density ratio in dB

    Returns:
    - BER: Bit Error Rate
    """
    n = Nbits
    R = 2000000
    fc = 8000000
    ns = nsamp
    T = 1 / R
    Ts = T / ns
    fss = 1 / Ts

    # Calculate SNR
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB

    # Generate random input bits and apply precoding
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y
    x[0] = 1
    for i in range(1, len(y)):
        x[i] = y[i] * x[i-1]  # Apply precoding rule

    # Generate NRZ polar pulse train samples
    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time grid
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    ## MSK TRANSMITTER
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)  # in-phase component
    xs_q = np.sin(theta)  # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid ts
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    # Updated MSK decoding for precoded bits with recursive logic
    bi = 1
    xr_1 = 1  # Initialize previous decoded bit (xr_1)
    xr = np.zeros(n)  # Array to store decoded bits

    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        
        # Matched filter output (to match MSK modulation characteristics)
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # In-phase matched filter pulse
        gmq = -gmi  # Quadrature matched-filter pulse (sin is negative of cosine)
        
        # Save previous in-phase matched filter output
        bi_1 = bi
        
        # Decode in-phase (I) and quadrature (Q) components
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        
        # Apply recursive decoding rule for precoded MSK
        xr[k] = bi_1 * bq  # Decode the k-th bit
        xr[k+1] = bi * bq  # Decode the (k+1)-th bit
        
        # Update the previously decoded bit (xr_1)
        xr_1 = xr[k+1]

    xr = xr.reshape(-1)
    err = np.not_equal(y, xr)
    errors = np.sum(err)
    return 0.5*errors / Nbits

# Function to simulate BER and plot
def update_msk_plot(nsamp):
    """
    Update the BER plot for MSK modulation with and without precoding.

    Parameters:
    - nsamp: Oversampling factor
    """
    # Start timer and display loading animation
    if loader_html4.value != loading:
        loader_html4.value = loading  # Set loading indicator only if not already set
    start_time = time.time()

    EbNo_range = np.arange(0, 9, 1)  # EbNo from 0 to 10 dB
    Nbits = 10000  # Increase number of bits to reduce variance

    simulated_BER = []
    simulated_BER_precoding = []
    theoretical_BER = []
    theoretical_BER_precoded = []
    
    for EbNo in EbNo_range:
        # Simulate both with and without precoding
        sim_BER = msk_errors(Nbits, nsamp, EbNo)
        sim_BER_precoded = msk_errors_precoding(Nbits, nsamp, EbNo)
        
        # Theoretical BER without precoding
        theoretical_BER_value = 0.9 * erfc(np.sqrt(10**(EbNo / 10)))
        # Theoretical BER with precoding
        theoretical_BER_precoded_value = erfc(np.sqrt(10**(EbNo / 10))) / 2
        
        simulated_BER.append(sim_BER)
        simulated_BER_precoding.append(sim_BER_precoded)
        theoretical_BER.append(theoretical_BER_value)
        theoretical_BER_precoded.append(theoretical_BER_precoded_value)

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbNo_range, simulated_BER, 'o', label='Simulated BER (without precoding)')
    plt.semilogy(EbNo_range, simulated_BER_precoding, 's', label='Simulated BER (with precoding)')
    plt.semilogy(EbNo_range, theoretical_BER, label='Theoretical BER (without precoding)')
    plt.semilogy(EbNo_range, theoretical_BER_precoded, label='Theoretical BER (with precoding)')
    plt.xlabel('$E_b/N_0$ (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('MSK Modulation (with and without precoding)')
    plt.legend()
    plt.grid(True, which='both')
    plt.show()

    # Update timer and loading indicator
    elapsed_time = time.time() - start_time
    timer_html4.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html4.value = done


# Interactive UI components
nsamp_slider = widgets.IntSlider(
    value=32,
    min=16,
    max=128,
    step=16,
    description='Oversampling Factor:',
    layout=Layout(width='auto', flex='1 1 auto'),
    style={'description_width': 'initial'}, 
    continuous_update=False
)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html4, timer_html4], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first refactored code)
input_widgets = widgets.VBox([nsamp_slider], layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the update_msk_plot function
out = widgets.interactive_output(update_msk_plot, {'nsamp': nsamp_slider})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()

### Step 2:

For the data of the previous question, find the values of the parameters of an
equivalent (in terms of bit rate) QPSK system. Compare the two systems in
terms of BER and bandwidth.

```{tip}
According to the figure below, for each odd k, the new bi (valid from the
next even k) and bq are calculated. Based on these, the two new bits are calculated,
xr(k)=bi_1*bq and xr(k+1)=bi*bq. With precoding, bit xr_1 of the previous period (k-1) is
also used.
```

![lab6_1.png](../content/images/lab6_1.png)
